# Xerrada Ansible - Python Girona

----

Jaume Flórez Valenzuela
(amb la colaboració de Manel Clos)

# Què és Ansible

- Ansible és una eina que ens permet instalar i actualitzar paquets remotament.
- Ansible és una aplicació server-side, és a dir, que no l'hem d'instalar a tots els clients.
- Ansible és un paquet de python, per utilitzar-lo únicament necessitem fer `pip install ansible`

# Idempotència

Una vegada instalat tot el playbook, s'hauria de poder executar de nou sense realitzar cap canvi.
Això ens permet actualitzacions/instalacions massives amb diferents entorns.

Es basa amb el terme matemàtic on identifiquem que una operació és indempotent quan
en aplicarla a un terme, el resultat és el mateix que el terme.

>“_En matemática y lógica, la idempotencia es la propiedad para realizar una acción determinada varias veces y aun así conseguir el mismo resultado que se obtendría si se realizase una sola vez_”

# Tipus de Fitxers

- **Inventaris**: Contenen informació sobre els diferents hosts on aplicar les tasques.
- **Variables**: Fitxers on podem definir un seguit de variables per utilitzar en les tasques.
- **Playbook**: Fitxer on declarem les diferents tasques a realitzar.

# Estructura recomanada (I)
```
production                # inventory file for production servers
staging                   # inventory file for staging environment

group_vars/
   group1                 # here we assign variables to particular groups
   group2                 # ""
host_vars/
   hostname1              # if systems need specific variables, put them here
   hostname2              # ""

library/                  # if any custom modules, put them here (optional)
module_utils/             # if any custom module_utils to support modules, put them here (optional)
filter_plugins/           # if any custom filter plugins, put them here (optional)

site.yml                  # master playbook
webservers.yml            # playbook for webserver tier
dbservers.yml             # playbook for dbserver tier

```

# Estructura recomanada (II)

```
roles/
    common/               # this hierarchy represents a "role"
        tasks/            #
            main.yml      #  <-- tasks file can include smaller files if warranted
        handlers/         #
            main.yml      #  <-- handlers file
        templates/        #  <-- files for use with the template resource
            ntp.conf.j2   #  <------- templates end in .j2
        files/            #
            bar.txt       #  <-- files for use with the copy resource
            foo.sh        #  <-- script files for use with the script resource
        vars/             #
            main.yml      #  <-- variables associated with this role
        defaults/         #
            main.yml      #  <-- default lower priority variables for this role
        meta/             #
            main.yml      #  <-- role dependencies
        library/          # roles can also include custom modules
        module_utils/     # roles can also include custom module_utils
        lookup_plugins/   # or other types of plugins, like lookup in this case
```

# Estructura dels inventaris (I)


## Que hi definim?

- **Hosts**: Utilitzant el _hostname_ o la seva _direcció IP_
- **Grups**: Agrupen hosts
- **Variables**: tant per un host o per un grup


# Estructura dels inventaris (II)

## Per a què serveixen els grups?

Els Grups ens permeten:

- Definir tasques (playbooks) específics per grups
- Definir variables específiques per grups
- Agrupar hosts segons "responsabilitats" o tipologia

Un Host sempre pertany com a mínim al grup "all".

Si un Host no té grup (a part del "all") s'afegeix al grup "ungrouped".

# Estructura dels inventaris (III)

##  Format clàssic (definició)

1. A no ser que la línia estigui enclausada amb ‘`[ ]`’, es tractarà com a host.
2. Si contenen únicament text entre '`[ ]`', representen un grup i tot el que segueix fins trobar un altre '`[ ]`' seran hosts d'aquest grup.
3. Si apart del text, conté el caràcter '`:`', representen atributs d'un grup, que seràn:
  - VARS:     Variables del grup, definides amb el format "`<variable>: <valor>`"
  - CHILDREN: Defineix quins grups són "fills" del grup actual
4. Es poden vincular variables a un host amb el format "`<variable>=<valor>`" separant les diferents variables amb espais.
  

# Estructura dels inventaris (III)

##  Format clàssic (exemple)

```
[web]
servidor_web  192.168.1.100
[documentacio]
docs_ansible  192.168.1.101  ansible_ssh_user=ansible src=/home/ansible/src
[web:children]
documentacio
[web:vars]
ansible_ssh_user: webmaster
```

# Estructura dels inventaris (IIII)

##  Format YAML (definició)

Utilitzarem el format d'arbre característic de YAML

```yaml
group:
    hosts:
        <host>:
            <nom_variable>: <valor_variable>
    children:
        group:
            ...
    vars:
        <nom_variable>: <valor_variable>
```



# Estructura dels inventaris (IIII)

##  Format YAML (exemple)

```yaml
web:
    hosts:
        servidor_web: 192.168.1.100
    children:
        documentacio:
            hosts:
                docs_ansible: 192.168.1.101
                    ansible_ssh_user: ansible
                    src: /home/ansible/src
    vars:
        ansible_ssh_user: webmaster
```